# Crashes to JSON

Use `Data Query Tool` to create a new export from UCONN Crashes (Town=Hartford, last five years): https://www.ctcrash.uconn.edu/QueryTool2.action

This notebook combines data from two tables (0 and 2) into a small JSON file, which is an array of the following objects:
```
{
    "x":41.73936,
    "y":-72.66404,
    "d":1593820800,
    "p":0,
    "c":0
}
```

* `d` is timestamp that needs to be multiplied by `100,000` for a proper JS timestamp
* `p` is a flag for pedestrians (1=true, 0=false)
* `c` is a flag for cyclists
* `id` is crash ID
* `r` is route class (1-Interstate, 2-USRoute, 3-State, 4-Local)
* `s` is crash severity (O for no injuries, A for any injury, and K for fatal crash)

In [66]:
import pandas as pd

In [67]:
export = '70836'

### Import roles from table 2

In [68]:
roles = pd.read_csv(
    f'export_{export}/export_{export}_2.csv',
    skiprows=1,
    usecols=['CrashId', 'Person Type Text Format']
)

roles = roles.groupby(['CrashId'])['Person Type Text Format'].unique()

### Import crashes from table 0

In [69]:
crashes = pd.read_csv(
    f'export_{export}/export_{export}_0.csv',
    skiprows=1,
    usecols=['CrashId', 'Latitude', 'Longitude', 'Date Of Crash', 'Time of Crash', 'Crash Severity', 'Route Class']
)

### Merge data and save to .json

In [32]:
crashes.columns = ['id', 'x', 'y', 'd', 't', 's', 'r']

crashes.d = crashes.d.apply(lambda x: int(pd.to_datetime(x).timestamp() / 100))
crashes.t = crashes.t.apply(lambda x: x[:-3])

crashes['p'] = crashes.id.apply(lambda x: 1 if x in roles and 'Pedestrian' in roles[x] else 0)
crashes['c'] = crashes.id.apply(lambda x: 1 if x in roles and 'Bicyclist' in roles[x] else 0)

In [33]:
crashes.to_json('data/crashes.json', orient='records', double_precision=5)

In [88]:
crashes[crashes.CrashId == 743727]

,CrashId,Latitude,Longitude,Date Of Crash,Time of Crash,Crash Severity,Route Class
35595,743727,41.761163,-72.675626,2020-01-22,10:00:00,O,0


In [89]:
crashes['Route Class'].value_counts()

4    24613
1     8680
3     2086
2     1899
0      888
Name: Route Class, dtype: int64